<a href="https://colab.research.google.com/github/951237/TIL/blob/master/TIL_%ED%81%AC%EB%A1%A4%EB%A7%81_%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94%ED%8F%89%EC%A0%90_200327.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 네이버 영화평점 데이터 크롤링

In [0]:
from urllib import request
from bs4 import BeautifulSoup as soup

class Review:
    def __init__(self, comment, date, star, good, bad):
        self.comment = comment
        self.date = date
        self.star = star
        self.good = good
        self.bad = bad

    def show(self):
        print("내용 : " + self.comment +
              "\n날짜 : " + self.date +
              "\n별점 : " + self.star +
              "\n좋아요 : " + self.good +
              "\n싫어요 : " + self.bad)


# URL = 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=189001'


def get_html(url):
    html = request.urlopen(url)
    bs_obj = soup(html, "html.parser")
    print('페이지 소스 로딩 완료!\n')
    return bs_obj


def crawl(soup):
    review_list = []
    # title = soup.find('h3', {'class' : 'h_movie'}).find('a').text
    title = soup.find('h3', class_='h_movie').find('a').text  # 영화제목 추출
    div = soup.find('div', class_='score_result')

    data_list = div.select("ul > li")  # 평정 영역의 한줄평 선택

    for review in data_list:
        star = review.find('div',
                           class_='star_score').text.strip()  # strip() 빈공간 여백 삭제
        reply = review.find('div', class_='score_reple')  # 리뷰 선택하기
        comment = reply.find("p").text.strip()  # 리뷰 내용
        date = reply.select("dt > em")[1].text.strip()  # 리뷰 날짜
        button = review.find('div', class_='btn_area')  # 버튼 영역 선택
        sympathy = button.select("a > strong")  # 리뷰의 공감과 비공감 선택 태그 부분 선
        good = sympathy[0].text  # 좋아요 수치
        bad = sympathy[1].text  # 싫어요 수치
        review_list.append(Review(comment, date, star, good, bad))
    return title, review_list


# soup = get_html(URL)
# title, review_list = crawl(soup) #제목과 크롤링한 리뷰내용 리턴

# print(f'제목 : {title}')
# for review in review_list:
#     review.show()
#     print()


# 데이터 프레임화

In [0]:
import pandas as pd
import time

def get_summary(review_list):
    star_list = [] # 별점 담기
    good_list = [] # 좋아요 점수 담기
    bad_list = [] # 싫어요 점수 담기

    for review in review_list:
        star_list.append(int(review.star))  # 영화 별점 담기
        good_list.append(int(review.good))  # 영화 좋아요 점수 담기
        bad_list.append(int(review.bad))   # 영화 싫어요 점수 담기

    star_series = pd.Series(star_list)  # 영화 별점을 데이터시리즈로 바꾸기
    good_series = pd.Series(good_list)  # 영화 좋아요 점수를 시리즈로 바꾸기
    bad_series = pd.Series(bad_list)    # 영화 실헝요 점수를 시리즈로 바꾸기

    summary = pd.DataFrame({        # 중괄호 유의
        'Star' : star_series,       # star 칼럼 만들기
        'Good' : good_series,       # Good 칼럼 만들기
        'Bad' : bad_series,         # Bad 칼럼 만들기
        'Score' : good_series / (good_series + bad_series)  # Score 칼럼 만들기
    })
    
    return summary

## 다수의 영화 데이터 시각화

In [4]:
movie_code_list = [189001, 187321, 190728, 136900, 167657, 174321, 184859]  # 영화코드 리스트에 담기
review_lists = []   # 영화 리뷰를 담을 리스트

BASE_URL = 'https://movie.naver.com/movie/bi/mi/basic.nhn?code='    # 영화 코드와 합칠 베이스 url

for i in movie_code_list:   
    URL = BASE_URL + str(i)     # url 주소 생성하기
    bs_obj = get_html(URL)      # beautiful 객체 생성
    title, review_list = crawl(bs_obj)  # 객체 내에서 크롤링
    summary = get_summary(review_list)  # 리뷰의 별점, 좋아요, 싫어요 점수 데이터 프레임 만들기
    print(f'{title}')           # 영화제목 출력
    print(summary,'\n')         # 영화 데이터 프레임 출력
    review_lists.append((title, review_list))   # 제목과 리뷰를 리스트에 담기



페이지 소스 로딩 완료!

인비저블맨
   Star  Good  Bad     Score
0    10  3896  791  0.831235
1     8  1889  159  0.922363
2     8  1381  106  0.928716
3     8   748  107  0.874854
4     7   711  182  0.796193 

페이지 소스 로딩 완료!

1917
   Star  Good  Bad     Score
0    10  2359  145  0.942093
1     8  1615  149  0.915533
2    10  1304   88  0.936782
3    10  1055   50  0.954751
4    10  1043   64  0.942186 

페이지 소스 로딩 완료!

다크 워터스
   Star  Good  Bad     Score
0     6   409   82  0.832994
1    10   182   14  0.928571
2    10   217   55  0.797794
3    10   164   14  0.921348
4    10   133    7  0.950000 

페이지 소스 로딩 완료!

어벤져스: 엔드게임
   Star   Good  Bad     Score
0    10  23129  650  0.972665
1    10  15286  432  0.972516
2    10  14896  463  0.969855
3    10  12871  528  0.960594
4    10  11550  423  0.964671 

페이지 소스 로딩 완료!

명탐정 피카츄
   Star  Good  Bad     Score
0    10  1928  105  0.948352
1    10   973   84  0.920530
2    10   831   71  0.921286
3     8   942  283  0.768980
4     8   617   42  0.936267 

페이

In [0]:
review_lists

## 글꼴 설치하기

In [0]:
!apt-get update -qq
!apt-get install fonts-nanum* -qq

## 시각화 하기

In [0]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm    # 글꼴 매니저

font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'  #글꼴 경로
fontprop = fm.FontProperties(fname=font_path, size=18)  # 글꼴 속성

In [0]:
def movie_compare(review_lists):    # 영화 리뷰 별점을 시각화 하기
    count = 1   # 영화 제목 카운트
    x = []      # 영화 숫자 카운트 저장
    y = []      # 별점 평균 저장
    for movie, review_list in review_lists:
        x.append(x)     # 영화 숫자 저장
        summary = get_summary(review_list)      # 리뷰 데이터 프레임 만들기
        summary = summary[summary['Score'] > 0.8]   # 점수가 0.8 이상만 모으기
        y.append(summary['Star'].mean())        # y리스트에 별점 평균 누적하기
        count += 1      # 영화숫자 카운트
    
    plt.bar(x,y)
    plt.title('영화 별점 비교', fontproperties = fontprop)
    plt.xlabel('영화 번호', fontproperties = fontprop)
    plt.ylabel('신뢰성 별점 평균', fontproperties = fontprop)
    plt.show()

movie_compare(review_lists)